# Подключение библиотек

In [55]:
#!pip install catboost

In [56]:
import numpy as np
import pandas as pd
import catboost
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка данных

In [57]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [58]:
data_path = "/content/drive/MyDrive/intern_task.csv"
data = pd.read_csv(data_path)
data.head()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
0,0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,0.0,0.000000,0.454545,0.890238,8.655534,1.000000,0.077778,0.002222,1.0,0.333333
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.773976,23.130514,0.000000,0.027826,0.000430,44.0,14.666667
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,0.0,0.000000,0.000000,0.918308,13.351339,0.000000,0.014925,0.000104,22.0,7.333333
3,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.975355,18.240926,0.000000,0.053140,0.000255,8.0,2.666667
4,2,10,3.0,0.0,3.0,1.0,3.0,1.000000,0.0,1.000000,...,273.0,79.670665,0.200000,0.990119,31.786048,0.333333,0.046512,0.000307,24.0,8.000000


In [59]:
features = list(data.columns[1::])

Обработаем пропуски в данных

In [60]:
data = data.dropna()

In [61]:
print(max(data['rank']))

4


In [62]:
sorted_df = data.sort_values(['query_id', 'rank'], ascending=[True, False])
sorted_df.tail(10)

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
235212,0,29995,1.0,0.0,1.0,1.0,1.0,0.5,0.0,0.5,...,0.0,0.0,0.166667,0.471409,13.302685,0.5,0.007937,0.000051,16.0,8.0
235226,0,29995,1.0,0.0,1.0,1.0,1.0,0.5,0.0,0.5,...,0.0,0.0,0.100000,0.471409,26.605371,0.5,0.019608,0.000290,13.0,6.5
235231,0,29995,1.0,1.0,1.0,1.0,1.0,0.5,0.5,0.5,...,0.0,0.0,0.125000,0.471409,13.302685,0.5,0.013682,0.000165,41.0,20.5
235232,0,29995,1.0,1.0,1.0,1.0,1.0,0.5,0.5,0.5,...,1.0,3.2,0.142857,0.471409,13.302685,0.5,0.015284,0.000087,4.0,2.0
235233,0,29995,1.0,0.0,1.0,0.0,1.0,0.5,0.0,0.5,...,0.0,0.0,0.000000,0.471409,26.605371,0.0,0.011521,0.000109,13.0,6.5
235234,0,29995,1.0,0.0,0.0,0.0,1.0,0.5,0.0,0.0,...,0.0,0.0,0.000000,0.471409,0.000000,0.0,0.001536,0.000002,2.0,1.0
235235,0,29995,1.0,0.0,0.0,0.0,1.0,0.5,0.0,0.0,...,7.0,25.6,0.000000,0.471409,0.000000,0.0,0.000535,0.000000,1.0,0.5
235239,0,29995,1.0,0.0,0.0,0.0,1.0,0.5,0.0,0.0,...,8.0,35.3,0.000000,0.471409,0.000000,0.0,0.000499,0.000000,1.0,0.5
235243,0,29995,1.0,0.0,0.0,0.0,1.0,0.5,0.0,0.0,...,0.0,0.0,0.000000,0.471409,0.000000,0.0,0.004732,0.000023,6.0,3.0
235249,0,29995,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.076923,0.000000,0.000000,0.5,0.006410,0.000000,0.0,0.0


Разделим данные на тренировочную и тестовую выборку

In [63]:
train_data, val_data = train_test_split(data, test_size = 0.2)
print(f"Размер тренировочной выборки: {len(train_data)}")
print(f"Размер тестовой выборки: {len(val_data)}")

Размер тренировочной выборки: 188206
Размер тестовой выборки: 47052


# Применение Catboost.Classifier

In [64]:
train_features = train_data[features]
train_target = train_data['rank']
val_features = val_data[features]
val_target = val_data['rank']

In [65]:
model = catboost.CatBoostClassifier()
# model.fit(train_features, train_target)

In [ ]:
predictions = model.predict(val_features)
preds = np.array([predictions[i][0] for i in range(len(predictions))])
print(preds)
print(val_target.to_numpy())
ndcg = ndcg_score([val_target.to_numpy()], [preds], k=5)
print(f"NDCG_5: {ndcg}")

[1 0 0 ... 0 0 2]
[2 1 0 ... 0 0 1]
NDCG_5: 0.7574626865671641


# Применение Catboost.Regressor

В предыдущем пункте наше решение основывалось на полном доверии к классификатору, теперь рассмторим другой подход

Будем действовать по следующему алгортму:

1) Расставим релевантность докумета запросу. Переведя "категориальные" позиции из {0, 1, 2, 3, 4} в удобные для регрессии вещественные из [0, 1] так, чтобы при максимальной "релевантности" получали минимальную позицию и наоборот

2) Применим ```Regressor``` $\rightarrow$ получим числа соответствующие "релевантоности" документа запросу, то есть чем полученное число больше, тем документ подходит запросу качесвтеннее

3) Разобьем документы на группы согласно полученному значению релевантности (сделаем это равномерно для пяти позиций)

Таким образом, получим модель ранжирования, чем выше "релевантность", тем лучше документ подходит запросу

Соотнесем ```rank``` и релевантность

In [66]:
rank_to_relevance = {0: 1, 1: 0.75, 2: 0.5, 3: 0.25, 4: 0}
data['relevance'] = data['rank'].map(rank_to_relevance)
data.head()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,relevance
0,0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,0.000000,0.454545,0.890238,8.655534,1.000000,0.077778,0.002222,1.0,0.333333,1.00
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.000000,0.000000,0.773976,23.130514,0.000000,0.027826,0.000430,44.0,14.666667,0.75
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,0.000000,0.000000,0.918308,13.351339,0.000000,0.014925,0.000104,22.0,7.333333,1.00
3,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.000000,0.000000,0.975355,18.240926,0.000000,0.053140,0.000255,8.0,2.666667,0.75
4,2,10,3.0,0.0,3.0,1.0,3.0,1.000000,0.0,1.000000,...,79.670665,0.200000,0.990119,31.786048,0.333333,0.046512,0.000307,24.0,8.000000,0.50


Применим ```Regressor```

In [67]:
train_data, val_data = train_test_split(data, test_size = 0.2)

train_features = train_data[features]
train_target = train_data['relevance']
val_features = val_data[features]
val_target = val_data['relevance']

model = catboost.CatBoostRegressor()
# model.fit(train_features, train_target)

Теперь запишем каждому документу предсказание модели

In [49]:
predictions = model.predict(val_features)
val_data['predictions'] = predictions
val_data.head()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,relevance,predictions
149549,2,18580,2.0,0.0,2.0,0.0,2.0,1.000000,0.000000,1.000000,...,0.000000,0.966713,10.243247,0.0,0.036891,0.000874,54.0,27.000000,0.50,0.789156
53089,1,7435,3.0,2.0,3.0,3.0,3.0,1.000000,0.666667,1.000000,...,0.250000,0.960613,23.298098,1.0,0.017644,0.000018,29.0,9.666667,0.75,0.735281
144185,0,18040,3.0,0.0,0.0,0.0,3.0,1.000000,0.000000,0.000000,...,0.000000,0.980481,0.000000,0.0,0.010612,0.000006,17.0,5.666667,1.00,0.896252
115230,1,14710,1.0,0.0,1.0,1.0,1.0,0.500000,0.000000,0.500000,...,0.166667,0.119823,7.085005,0.5,0.030000,0.000625,9.0,4.500000,0.75,0.787514
118012,0,14965,1.0,0.0,1.0,0.0,1.0,0.333333,0.000000,0.333333,...,0.000000,0.548727,7.252768,0.0,0.008000,0.000113,11.0,3.666667,1.00,0.897390


In [53]:
val_data['pred_rank'] = val_data['predictions'].apply(lambda x: 0 if x >= 0.8 else (1 if 0.6 <= x < 0.8 else (2 if 0.4 <= x < 0.6 else (3 if 0.2 <= x < 0.4 else 4))))
val_data.head()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,relevance,predictions,pred_rank
149549,2,18580,2.0,0.0,2.0,0.0,2.0,1.000000,0.000000,1.000000,...,0.966713,10.243247,0.0,0.036891,0.000874,54.0,27.000000,0.50,0.789156,1
53089,1,7435,3.0,2.0,3.0,3.0,3.0,1.000000,0.666667,1.000000,...,0.960613,23.298098,1.0,0.017644,0.000018,29.0,9.666667,0.75,0.735281,1
144185,0,18040,3.0,0.0,0.0,0.0,3.0,1.000000,0.000000,0.000000,...,0.980481,0.000000,0.0,0.010612,0.000006,17.0,5.666667,1.00,0.896252,0
115230,1,14710,1.0,0.0,1.0,1.0,1.0,0.500000,0.000000,0.500000,...,0.119823,7.085005,0.5,0.030000,0.000625,9.0,4.500000,0.75,0.787514,1
118012,0,14965,1.0,0.0,1.0,0.0,1.0,0.333333,0.000000,0.333333,...,0.548727,7.252768,0.0,0.008000,0.000113,11.0,3.666667,1.00,0.897390,0


Считаем метрику

In [54]:
ndcg = ndcg_score([val_data['rank']], [val_data['pred_rank']], k=5)
print(f"NDCG_5: {ndcg}")

NDCG_5: 0.7894736842105263


# Итог

Таким образом, нами было предложено два метода по ранжированию документов:

**1)** Использование catboost.Classifier, полученное значение метрики на валиационной выборке составило **0.76**

**2)** Метод, использующий регрессию, основанный на получении "релевантностей" документов, на валидационной выборке значение метрики получилось **0.79**

Лучшее решение - второй метод, **финальное качество: 0.79**